<a href="https://colab.research.google.com/github/vapaul10/Portfolio-of-Valeri/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web scraping using different libraries and modules

Make a data pipeline

Make a predictive model

In [54]:
import pandas as pd
import time

article = pd.read_html('https://en.wikipedia.org/wiki/2018_FIFA_World_Cup',header=0)
df = article[5]
count = 0
for i in range(0,len(article)):
  df = article[i]
  #print(df.head())
  count += 1
count

101

In [55]:
article[12]

,Uruguay,3–0,Russia
0,Suárez 10' Cheryshev 23' (o.g.) Cavani 90',Report,NaN


We have different approaches to get data from web:

1- To use request and urls

In [76]:
import bs4
import pandas as pd
import requests

In [57]:
def numeric_value(movie, tag, class_=None, order=None):
    if order:
        if len(movie.findAll(tag, class_)) > 1:
            to_extract = movie.findAll(tag, class_)[order]['data-value']
        else:
            to_extract = None
    else:
        to_extract = movie.find(tag, class_)['data-value']

    return to_extract


def text_value(movie, tag, class_=None):
    if movie.find(tag, class_):
        return movie.find(tag, class_).text
    else:
        return


def nested_text_value(movie, tag_1, class_1, tag_2, class_2, order=None):
    if not order:
        return movie.find(tag_1, class_1).find(tag_2, class_2).text
    else:
        return [val.text for val in movie.find(tag_1, class_1).findAll(tag_2, class_2)[order]]


def extract_attribute(soup, tag_1, class_1='', tag_2='', class_2='',
                      text_attribute=True, order=None, nested=False):
    movies = soup.findAll('div', class_='lister-item-content')
    data_list = []
    for movie in movies:
        if text_attribute:
            if nested:
                data_list.append(nested_text_value(movie, tag_1, class_1, tag_2, class_2, order))
            else:
                data_list.append(text_value(movie, tag_1, class_1))
        else:
            data_list.append(numeric_value(movie, tag_1, class_1, order))

    return data_list


In [58]:
url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'
def get_page_contents(url):
  page = requests.get(url, headers={"Accept-Language": "en-US"})
  return bs4.BeautifulSoup(page.text, "html.parser")

soup = get_page_contents(url)

In [77]:
titles = extract_attribute(soup, 'a')
release = extract_attribute(soup, 'span', 'lister-item-year text-muted unbold')
audience_rating = extract_attribute(soup, 'span', 'certificate')
runtime = extract_attribute(soup, 'span', 'runtime')
genre = extract_attribute(soup, 'span', 'genre')
imdb_rating = extract_attribute(soup, 'div', 'inline-block ratings-imdb-rating', False)
votes = extract_attribute(soup, 'span' , {'name' : 'nv'}, False, 0)
earnings = extract_attribute(soup, 'span' , {'name' : 'nv'}, False, 1)
#directors = extract_attribute(soup, 'p', '', 'a', '', True, 0, True)
actors = extract_attribute(soup, 'p', '', 'a', '', True, slice(1, 5, None), True)


df_dict = {'Title': titles, 'Relase': release, 'Audience Rating': audience_rating,
           'Runtime': runtime, 'Genre': genre, 'IMDB Rating': imdb_rating,
           'Votes': votes, 'Box Office Earnings': earnings, 
           'Actors': actors} #'Director': directors,
df = pd.DataFrame(df_dict)
df

,Title,Relase,Audience Rating,Runtime,Genre,IMDB Rating,Votes,Box Office Earnings,Actors
0,The Shawshank Redemption,(1994),R,142 min,\nDrama,\n\n9.3\n,"2,618,720","2,618,720",[]
1,The Godfather,(1972),R,175 min,"\nCrime, Drama",\n\n9.2\n,"1,813,390","1,813,390",[]
2,The Dark Knight,(2008),PG-13,152 min,"\nAction, Crime, Drama",\n\n9.0\n,"2,590,650","2,590,650",[]
3,The Lord of the Rings: The Return of the King,(2003),PG-13,201 min,"\nAction, Adventure, Drama",\n\n9.0\n,"1,796,877","1,796,877",[]
4,Schindler's List,(1993),R,195 min,"\nBiography, Drama, History",\n\n9.0\n,"1,330,239","1,330,239",[]
...,...,...,...,...,...,...,...,...,...
95,Ayla: The Daughter of War,(2017),None,125 min,"\nBiography, Drama, History",\n\n8.3\n,"39,374","39,374",[]
96,Dangal,(2016),Not Rated,161 min,"\nAction, Biography, Drama",\n\n8.3\n,"186,015","186,015",[]
97,Drishyam,(2013),Not Rated,160 min,"\nCrime, Drama, Thriller",\n\n8.3\n,"40,835","40,835",[]
98,The Hunt,(2012),R,115 min,\nDrama,\n\n8.3\n,"323,604","323,604",[]


In [78]:
imdb_rating[0].split()

['9.3']

In [79]:
df.to_csv('table')

In [80]:
base_url = requests.get('https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start=', timeout=5)
print(base_url.text)

<!DOCTYPE html><html lang="en" dir="ltr" data-reactroot=""><head><meta charSet="utf-8"/><link rel="preconnect" href="https://fonts.gstatic.com/" crossorigin=""/><link href="https://fonts.googleapis.com/css?family=Noto+Sans:400,400i,700,700i&amp;display=swap" rel="stylesheet"/><script defer="" src="https://d3hbwax96mbv6t.cloudfront.net/turnstile/s/943d9ca/turnstile-compiled.js" crossorigin="anonymous"></script><script defer="" src="/cmp/-/s/f7ee96ceca7a435f/runtime.js" crossorigin="anonymous"></script><script defer="" src="/cmp/-/s/14e8845a9c05f417/vendor.js" crossorigin="anonymous"></script><script defer="" src="/cmp/-/s/145a6cbc5e32ce53/reviews-aurora.js" crossorigin="anonymous"></script><link rel="stylesheet" href="/cmp/-/s/4984e033fd026eab/reviews-aurora.css"/><meta name="description" content="4,177 reviews from Google employees about Google culture, salaries, benefits, work-life balance, management, job security, and more."/><link rel="canonical" href="https://www.indeed.com/cmp/Go

In [63]:
def parse(full_url):
  page_content = BeautifulSoup(full_url.content, 'lxml')
  containers = page_content.findAll('div',
                                    {'class':'cmp-Review-container'})
  df = pd.DataFrame(columns = 
                    ['rating', 'rating_title',  'rating_description',
                     'rating_pros', 'rating_cons'])
  for item in containers:
    try:
      rating = item.find('div',
                         {'class': 'cmp-ReviewRating-text'}).text.replace('\n', '')
    except:
      rating = None
    try:
      rating_title = item.find('div',
                               {'class': 'cmp-Review-title'}).text.replace('\n', '')
    except:
        rating_title = None
    try:
      rating_description = item.find('span',
                                     {'itemprop': 'reviewBody'}).text.replace('\r', '. ')
    except:
      rating_description = None
    try:
      rating_pros = item.find('div', 
                          {'class': 'cmp-ReviewProsCons-prosText'}).text.replace('\n', '')
    except:
      rating_pros = None
    try:
      rating_cons = item.find('div',
      {'class': 'cmp-ReviewProsCons-consText'}).text.replace('\n', '')
    except:
      rating_cons = None
    df = df.append({'rating': rating,
                    'rating_title': rating_title,
                    'rating_description': rating_description,
                    'rating_pros': rating_pros, 
                    'rating_cons': rating_cons}, ignore_index=True)
  return df


In [64]:
def parse(full_url):
    page_content = BeautifulSoup(full_url.content, 'lxml')
    containers = page_content.findAll('div', 
                 {'class':'cmp-Review-container'})
    df = pd.DataFrame(columns = 
         ['rating', 'rating_title',  'rating_description',
                         'rating_pros', 'rating_cons'])
    
    for item in containers:        
        try:
            rating = item.find('div', 
                     {'class': 'cmp-ReviewRating-text'}).text.replace('\n', '')
        except:
            rating = None
        try:
            rating_title = item.find('div', 
                           {'class': 'cmp-Review-title'}).text.replace('\n', '')
        except:
            rating_title = None
        try:
            rating_description = item.find('span', 
                                 {'itemprop': 'reviewBody'}).text.replace('\r', '. ')
        except:
            rating_description = None
        try:
            rating_pros = item.find('div', 
                          {'class': 'cmp-ReviewProsCons-prosText'}).text.replace('\n', '')
        except:
            rating_pros = None
        try:
            rating_cons = item.find('div', 
                          {'class': 'cmp-ReviewProsCons-consText'}).text.replace('\n', '')
        except:
            rating_cons = None
        df = df.append({'rating': rating, 
             'rating_title': rating_title, 
             'rating_description': rating_description,
             'rating_pros': rating_pros, 
             'rating_cons': rating_cons}, ignore_index=True)
    return df

In [65]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [66]:
# base_url = 'https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start='

# all_reviews_df = pd.DataFrame(columns = ['rating', 'rating_title', 
# 'rating_description','rating_pros', 'rating_cons'])

# num_reviews = 20

# you can adjust this number on how many reviews you which to scrap
# while num_reviews < 3000:
#   full_url = base_url + str(num_reviews)
  
#   get_url = requests.get(full_url, timeout=5)
  
#   partial_reviews_df = parse(get_url)
  
#   all_reviews_df = all_reviews_df.append(
#      partial_reviews_df, ignore_index=True)
  
#  num_reviews += 20

In [67]:
# Retype in Code
base_url = 'https://www.indeed.com/cmp/Google/reviews?fcountry=ALL&start='

all_reviews_df = pd.DataFrame(columns= ['rating', 'rating_title',
'rating_description', 'rating_pros', 'rating_cons'])

num_reviews = 20

# you can adjust this number on how many reviews you which to scrape
while num_reviews < 3000:
  full_url = base_url + str(num_reviews)
  
  get_url = requests.get(full_url, timeout=5)
  partial_reviews_df = parse(get_url)
  
  all_reviews_df = all_reviews_df.append(partial_reviews_df, ignore_index=True)
  
  num_reviews += 20


In [68]:
all_reviews_df.to_csv('indeed_scrape.csv')

In [69]:
all_reviews_df

,rating,rating_title,rating_description,rating_pros,rating_cons


In [70]:
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import random

In [75]:
df = all_reviews_df

df.describe()

,rating,rating_title,rating_description,rating_pros,rating_cons
count,0,0,0,0,0
unique,0,0,0,0,0
top,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,NaN,NaN


In [ ]:
groups = df.groupby('rating').count()
Values = groups.rating_title
colors = ['r', 'g', 'b', 'c', 'm']
#making bar plot
plt.bar(([1,2,3,4,5]), Values, color= colors)
plt.title('Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Review Quantity')
plt.show()

In [ ]:
df['rating'][0]